In [ ]:
!pip install -U scikit-learn

     |████████████████████████████████| 6.8MB 3.3MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
#Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mutual_info_score
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import scale
from sklearn.utils import check_random_state
from sklearn.utils.fixes import _astype_copy_false
from sklearn.utils.validation import check_array, check_X_y
from sklearn.utils.validation import _deprecate_positional_args
from sklearn.utils.multiclass import check_classification_targets
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn import svm
from scipy import special, stats
from scipy.sparse import issparse
from sklearn.base import BaseEstimator
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import (as_float_array, check_array, check_X_y, safe_sqr,safe_mask)
from sklearn.utils.extmath import safe_sparse_dot, row_norms
from sklearn.utils.validation import check_is_fitted
from sklearn.utils.validation import _deprecate_positional_args
from sklearn.feature_selection._base import SelectorMixin
from sklearn.metrics import roc_auc_score
from scipy.sparse import issparse
from scipy.special import digamma
import scipy.stats as stats
from subprocess import check_output
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
#importing dataset
df = pd.read_csv('DLBCL.tab', delimiter='\t')
df.shape

(79, 7071)

In [ ]:
df=df.dropna().loc[1:,:]

In [ ]:
X = df.drop(['class'], axis=1)
X.head()

,A28102,AB000114_at,AB000115_at,AB000220_at,AB000409_at,AB000449_at,AB000450_at,AB000460_at,AB000462_at,AB000464_at,AB000466_at,AB000467_at,AB000468_at,AB000584_at,AB000895_at,AB000896_at,AB000897_at,AB000905_at,AB001106_at,AB001325_at,AB002314_at,AB002315_at,AB002318_at,AB002365_at,AB002366_at,AB002380_at,AB002382_at,AB002409_at,AB002559_at,AB003102_at,AB003103_at,AB003177_at,AB003698_at,AB004884_at,AB006190_at,AC000061_cds2_at,AC000061_cds3_at,AC000062_at,AC000064_cds1_at,AC000064_cds2_at,...,X53065_f_at,X64177_f_at,X67491_f_at,X71345_f_at,X97444_f_at,Z80780_f_at,X00351_f_at,X01677_f_at,M31667_f_at,L41268_f_at,X99479_f_at,HG658-HT658_f_at,M94880_f_at,S80905_f_at,X03068_f_at,Z34822_f_at,U87593_f_at,U88902_cds1_f_at,AC002076_cds2_at,D64015_at,HG2510-HT2606_at,L10717_at,L34355_at,L78833_cds4_at,M13981_at,M21064_at,M93143_at,S78825_at,U11863_at,U29175_at,U48730_at,U58516_at,U73738_at,X06956_at,X16699_at,X83863_at,Z17240_at,L49218_f_at,M71243_f_at,Z78285_f_at
2,-1,-45,176,97,-57,233,265,945,56,819,-526,-1075,569,-501,-185,-28,-132,77,564,897,-40,169,1277,-80,-20,117,684,-6,500,864,109,838,332,762,548,368,-51,48,231,102,...,353,357,69,233,371,-92,21210,13993,1645,354,-341,12542,8233,298,11319,-72,229,778,16,409,41,111,234,354,71,286,575,-210,-1468,165,132,1036,-161,1523,-72,897,366,3,76,-28
3,25,-17,531,353,122,155,209,1688,42,639,-368,-253,892,71,157,-101,-68,136,562,623,4,278,956,193,17,344,768,226,390,912,223,904,270,668,619,248,-59,-18,153,73,...,328,7216,-27,333,287,-31,7490,15232,411,272,258,17542,14754,270,5700,-136,48,346,13,338,2,708,185,363,221,176,409,-262,-2333,2076,147,4254,-75,6056,-68,737,81,-4,41,-45
4,73,91,257,80,614,507,760,2252,196,863,-618,-596,1963,-226,-237,-133,29,116,999,861,55,539,1245,44,-2,224,597,275,1270,1976,320,1171,479,986,762,412,-65,22,424,115,...,172,5885,84,237,400,281,24760,14082,384,233,28,13338,15725,433,9175,-273,96,335,-31,318,35,453,112,263,266,299,630,-399,-2024,1468,169,1934,-129,2901,-9,946,450,38,164,-62
5,267,41,202,138,198,355,245,1469,170,384,-698,-31,1150,202,-541,162,-55,113,549,434,-51,418,900,47,15,226,430,89,476,1164,192,517,287,634,674,404,-122,21,154,174,...,224,2606,59,196,188,61,18747,17627,347,206,67,16591,9551,160,8049,-214,33,269,-23,224,1,335,-85,93,126,63,166,-99,-626,910,13,2469,93,2465,-119,339,321,-63,73,-13
6,16,24,187,39,145,254,571,930,-11,439,-665,-306,1460,117,-679,-61,-82,110,838,651,-74,201,454,0,-37,698,390,-17,461,815,342,444,345,648,587,198,-102,6,269,151,...,206,1408,-1,226,226,402,7871,14335,727,203,66,12369,5489,307,7405,-134,177,335,-8,248,24,170,-24,146,370,188,822,-29,-2380,350,69,608,-39,1667,-87,721,213,15,155,-26


In [ ]:
# Mapping classes into integers
y = df['class']
y=y.map({'DLBCL': 1, 'FL': 0})
y.head()

2    1
3    1
4    1
5    1
6    1
Name: class, dtype: int64

In [ ]:
#Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((61, 7070), (61,), (16, 7070), (16,))

In [ ]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, SelectPercentile
constant_filter = VarianceThreshold(threshold=0.01)
constant_filter.fit(X_train)
X_train_filter = constant_filter.transform(X_train)
X_test_filter = constant_filter.transform(X_test)

In [ ]:
X_train_T = X_train_filter.T
X_test_T = X_test_filter.T
X_train_T = pd.DataFrame(X_train_T)
X_test_T = pd.DataFrame(X_test_T)

In [ ]:
X_train_T.duplicated().sum()

0

In [ ]:
duplicated_features = X_train_T.duplicated()

In [ ]:
features_to_keep = [not index for index in duplicated_features]
X_train_unique = X_train_T[features_to_keep].T
X_test_unique = X_test_T[features_to_keep].T
X_train_unique.shape, X_test_unique.shape

((61, 7070), (16, 7070))

In [ ]:
#Function for KNN Classifier C1
from sklearn.neighbors import KNeighborsClassifier
def run_knn(X_train, X_test, y_train, y_test):
    c1 =KNeighborsClassifier(n_neighbors=3)
    c1.fit(X_train, y_train)
    y_pred = c1.predict(X_train)
    y_test_pred=c1.predict(X_test)
    print('Training 80% data report::')
    print('Accuracy Score: ')
    print(accuracy_score(y_train, y_pred))
    print('F1-Score: ')
    print(f1_score(y_train, y_pred))
    print('Confusion Matrix: ')
    print(confusion_matrix(y_train, y_pred))
    print('Testing 20% data report::')
    print('Accuracy Score: ')
    print(accuracy_score(y_test, y_test_pred))
    print('F1-Score: ')
    print(f1_score(y_test, y_test_pred))
    print('Confusion Matrix: ')
    print(confusion_matrix(y_test, y_test_pred))


In [ ]:
#Function for SVM Classifier C2
def run_svm(X_train, X_test, y_train, y_test):
    c2 =svm.SVC()
    c2.fit(X_train, y_train)
    y_pred = c2.predict(X_train)
    y_test_pred=c2.predict(X_test)
    print('Training 80% data report::')
    print('Accuracy Score: ')
    print(accuracy_score(y_train, y_pred))
    print('F1-Score: ')
    print(f1_score(y_train, y_pred))
    print('Confusion Matrix: ')
    print(confusion_matrix(y_train, y_pred))
    print('Testing 20% data report::')
    print('Accuracy Score: ')
    print(accuracy_score(y_test, y_test_pred))
    print('F1-Score: ')
    print(f1_score(y_test, y_test_pred))
    print('Confusion Matrix: ')
    print(confusion_matrix(y_test, y_test_pred))

## Functions of Mutual Information based feature selection(f1), F- Classifier(f2) and T-Test(f3)

In [ ]:
# Functions of Mutual information
def _compute_mi(x, y, n_neighbors=3):
        return mutual_info_score(x, y)

def _iterate_columns(X, columns=None):
    if columns is None:
        columns = range(X.shape[1])

    if issparse(X):
        for i in columns:
            x = np.zeros(X.shape[0])
            start_ptr, end_ptr = X.indptr[i], X.indptr[i + 1]
            x[X.indices[start_ptr:end_ptr]] = X.data[start_ptr:end_ptr]
            yield x
    else:
        for i in columns:
            yield X[:, i]


def _estimate_mi(X, y,n_neighbors=3):
    X, y = check_X_y(X, y, accept_sparse='csc')
    n_samples, n_features = X.shape
    discrete_features = issparse(X)
    discrete_mask = np.empty(n_features, dtype=bool)
    discrete_mask.fill(discrete_features)
    
    mi = [_compute_mi(x, y, n_neighbors) for
          x, discrete_feature in zip(_iterate_columns(X), discrete_mask)]

    return np.array(mi)

def mutual_info_classifier(X, y, n_neighbors=3,):
    check_classification_targets(y)
    return _estimate_mi(X, y, n_neighbors)

In [ ]:
#Functions for F classifier 
def f_oneway(*args):
    n_classes = len(args)
    args = [as_float_array(a) for a in args]
    n_samples_per_class = np.array([a.shape[0] for a in args])
    n_samples = np.sum(n_samples_per_class)
    ss_alldata = sum(safe_sqr(a).sum(axis=0) for a in args)
    sums_args = [np.asarray(a.sum(axis=0)) for a in args]
    square_of_sums_alldata = sum(sums_args) ** 2
    square_of_sums_args = [s ** 2 for s in sums_args]
    sstot = ss_alldata - square_of_sums_alldata / float(n_samples)
    ssbn = 0.
    for k, _ in enumerate(args):
        ssbn += square_of_sums_args[k] / n_samples_per_class[k]
    ssbn -= square_of_sums_alldata / float(n_samples)
    sswn = sstot - ssbn
    dfbn = n_classes - 1
    dfwn = n_samples - n_classes
    msb = ssbn / float(dfbn)
    msw = sswn / float(dfwn)
    constant_features_idx = np.where(msw == 0.)[0]
    f = msb / msw #High variance upon low variance 
    # flatten matrix to vector in sparse case
    f = np.asarray(f).ravel()
    prob = special.fdtrc(dfbn, dfwn, f)
    return f, prob


def f_classif(X, y):
    X, y = check_X_y(X, y, accept_sparse=['csr', 'csc', 'coo'])
    args = [X[safe_mask(X, y == k)] for k in np.unique(y)]
    return f_oneway(*args)

In [ ]:
#Functions for  t-test 
import math
def t_oneway(*args):
    n_classes = len(args)
    args = [as_float_array(a) for a in args]
    n_samples_per_class = np.array([a.shape[0] for a in args])
    n_samples = np.sum(n_samples_per_class)
    ss_alldata = sum(safe_sqr(a).sum(axis=0) for a in args)
    sums_args = [np.asarray(a.sum(axis=0)) for a in args]
    square_of_sums_alldata = sum(sums_args) ** 2
    square_of_sums_args = [s ** 2 for s in sums_args]
    sstot = ss_alldata - square_of_sums_alldata / float(n_samples)
    ssbn = 0.
    for k, _ in enumerate(args):
        ssbn += square_of_sums_args[k] / n_samples_per_class[k]
    ssbn -= square_of_sums_alldata / float(n_samples)
    sswn = sstot - ssbn
    dfbn = n_classes - 1
    dfwn = n_samples - n_classes
    msb = ssbn / float(dfbn)
    msw = sswn / float(dfwn)
    constant_features_idx = np.where(msw == 0.)[0]
    t = (ssbn-sswn)**2/ (msb+msw) #Taking sqrt of t value instead of absolute as t due to limitation of numpy. t^2= (absolute differences of means/sum of variances)^2
    # flatten matrix to vector in sparse case
    t = np.asarray(t).ravel()
    prob = special.fdtrc(dfbn, dfwn, t)
    return t, prob


def t_test(X, y):
    X, y = check_X_y(X, y, accept_sparse=['csr', 'csc', 'coo'])
    args = [X[safe_mask(X, y == k)] for k in np.unique(y)]
    return t_oneway(*args)

In [ ]:
mi = mutual_info_classifier(X_train_unique, y_train) 
len(mi)

7070

In [ ]:
mi[: 10]

array([0.5527177 , 0.5527177 , 0.57544384, 0.5527177 , 0.5527177 ,
       0.52999156, 0.5527177 , 0.52999156, 0.52141373, 0.57544384])

In [ ]:
mi = pd.Series(mi)
mi.index = X_train_unique.columns

In [ ]:
mi.sort_values(ascending=False, inplace = True)

In [ ]:
mi.index

Int64Index([6288, 1903, 1874, 4738, 4737, 4736, 4735, 1880, 4732, 1883,
            ...
            2619, 5443, 5395, 4517, 6802, 4356, 6630, 5148,  127, 4261],
           dtype='int64', length=7070)

### Selecting N/3 Features of f1

In [ ]:
sel = SelectKBest(mutual_info_classifier, k=int(len(mi.index)/3*0.15)).fit(X_train_unique, y_train)#0.15 is multiplied to take 15%of the total features
X_train_unique.columns[sel.get_support()]

Int64Index([6084, 6090, 6091, 6093, 6097, 6098, 6099, 6100, 6101, 6102,
            ...
            7041, 7042, 7044, 7056, 7058, 7061, 7062, 7063, 7065, 7066],
           dtype='int64', length=353)

In [ ]:
len(X_train_unique.columns[sel.get_support()])

353

In [ ]:
X_train_mi = sel.transform(X_train_unique)
X_test_mi = sel.transform(X_test_unique)
X_train_mi.shape

(61, 353)

### Selecting N/3 Features of f2

In [ ]:
fc= f_classif(X_train, y_train) 
len(fc[0])

7070

In [ ]:
fc[: 10]

(array([1.86599787, 2.71660408, 0.66495707, ..., 0.32488098, 0.97306399,
        0.67826326]),
 array([0.17711867, 0.10462485, 0.41809516, ..., 0.57085316, 0.3279454 ,
        0.41350211]))

In [ ]:
#0.15 is multiplied to take 15% of total features
fvalue_selector = SelectKBest(f_classif, k =int(len(fc[0])/3*0.15) )

In [ ]:
features_kbest = fvalue_selector.fit_transform(X_train_unique, y_train)

In [ ]:
X_train_fc = fvalue_selector.transform(X_train_unique)
X_test_fc = fvalue_selector.transform(X_test_unique)
X_train_fc.shape

(61, 353)

### Selecting N/3 Features of f3

In [ ]:
tt= t_test(X_train, y_train) 
tt[: 10]

(array([2.96084110e+07, 1.89911571e+07, 9.28682629e+08, ...,
        1.84427939e+08, 9.96089617e+07, 1.72028268e+08]),
 array([7.04754182e-170, 3.44748233e-164, 5.04337723e-214, ...,
        2.58771365e-193, 2.01835523e-185, 2.01655147e-192]))

In [ ]:
#0.15 is multiplied to take 15% of total features
tvalue_selector = SelectKBest(t_test, k =int(len(fc[0])/3*0.15) )

In [ ]:
featurestt_kbest = tvalue_selector.fit_transform(X_train_unique, y_train)

In [ ]:
X_train_t = tvalue_selector.transform(X_train_unique)
X_test_t = tvalue_selector.transform(X_test_unique)
X_train_t.shape

(61, 353)

### Task B.1: Taking Union of f1, f2, f3

In [ ]:
from sklearn.pipeline import FeatureUnion, Pipeline
combined_features = FeatureUnion([("mi", sel), ("fc",fvalue_selector ),("tt",tvalue_selector)])

# Use combined features to transform dataset:
X_train_uni = combined_features.fit(X_train_unique, y_train).transform(X_train_unique)
X_test_uni = combined_features.fit(X_train_unique, y_train).transform(X_test_unique)

In [ ]:
X_train_uni.shape

(61, 1059)

## Task B.2: Application Of KNN (C1)

In [ ]:
%%time
run_knn(X_train_uni, X_test_uni, y_train, y_test)

Training 80% data report::
Accuracy Score: 
0.9344262295081968
F1-Score: 
0.9534883720930233
Confusion Matrix: 
[[16  0]
 [ 4 41]]
Testing 20% data report::
Accuracy Score: 
0.75
F1-Score: 
0.8333333333333333
Confusion Matrix: 
[[ 2  1]
 [ 3 10]]
CPU times: user 58.2 ms, sys: 1.06 ms, total: 59.3 ms
Wall time: 56.2 ms


## Task B.3: Application Of SVM (C2)

In [ ]:
%%time
run_svm(X_train_uni, X_test_uni, y_train, y_test)

Training 80% data report::
Accuracy Score: 
0.8524590163934426
F1-Score: 
0.9090909090909091
Confusion Matrix: 
[[ 7  9]
 [ 0 45]]
Testing 20% data report::
Accuracy Score: 
0.8125
F1-Score: 
0.896551724137931
Confusion Matrix: 
[[ 0  3]
 [ 0 13]]
CPU times: user 52.3 ms, sys: 3 ms, total: 55.3 ms
Wall time: 52.8 ms


## Task B.4: Comparison is done in separate file comparison.docx

### End of Task B